# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('myproj').getOrCreate()


data = spark.read.csv('/FileStore/tables/customer_churn.csv',inferSchema=True,header=True)

In [4]:
data.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [5]:
data.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
 Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites| Onboard_date| Location| Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
Cameron Williams|42.0| 11066.8| 0| 7.22| 8.0|2013-08-30 07:00:40|10265 Elizabeth M...| Harvey LLC| 1|
 Kevin Mueller|41.0| 11916.22| 0| 6.5| 11.0|2013-08-13 00:38:46|6157 Frank Garden...| Wilson PLC| 1|
 Eric Lozano|38.0| 12884.75| 0| 6.67| 12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...| 1|
 Phillip White|42.0| 8010.76| 0| 6.71| 10.0|2014-04-22 12:43:12|13120 Daniel Moun...| Smith Inc| 1|
 Cynthia Norton|37.0| 9191.58| 0| 5.56| 9.0|2016-01-19 15:31:15|765 Tricia Row Ka...| Love-Jones| 1|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [6]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|2006-01-02 04:16:13|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|2016-12-28 04:07:38|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+

In [7]:
data.groupBy('Names').count().show()

+------------------+-----+
 Names|count|
+------------------+-----+
 Patrick Bell| 1|
 Patrick Robinson| 1|
 Chelsea Marsh| 1|
 John Barber| 1|
 Amber Evans| 1|
 David Compton| 1|
 Mr. Jerome Dawson| 1|
 Lisa Davis| 1|
 Maria Stanley| 1|
Alexandra Phillips| 1|
 Nicholas Levy| 1|
 Richard Farmer| 1|
 Linda Hubbard| 1|
 Jesse Mitchell| 1|
 Brittany Green| 1|
 Timothy Johnston| 1|
 Charles Whitney| 1|
 Tony Schneider| 1|
 Stefanie Miranda| 1|
 Robert Joseph| 1|
+------------------+-----+
only showing top 20 rows

In [8]:
data.columns

Out[404]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [9]:
from pyspark.sql.functions import year

In [10]:
df02 = data.select('Age'
                   ,'Total_Purchase'
                   ,'Years'
                   ,'Num_Sites'
                   ,'Account_Manager'
                   ,'Churn'
                   ,year('Onboard_date').alias('year'))

In [11]:
# Não vale a pena transformar a coluna Company em variável discreta, pois de um total de 900 amostras, ela tem um total de 873 amostras distintas.

# Isso quer dizer que não tem muitos exemplos para deduzir e criar aprendizado pelos poucos exemplos que há. Tornando esta variável inviável para transformação e modelagem.

from pyspark.sql.functions import countDistinct

data.select(countDistinct('Company')).show()

+-----------------------+
count(DISTINCT Company)|
+-----------------------+
 873|
+-----------------------+

In [12]:
df02.show(5)

+----+--------------+-----+---------+---------------+-----+----+
 Age|Total_Purchase|Years|Num_Sites|Account_Manager|Churn|year|
+----+--------------+-----+---------+---------------+-----+----+
42.0| 11066.8| 7.22| 8.0| 0| 1|2013|
41.0| 11916.22| 6.5| 11.0| 0| 1|2013|
38.0| 12884.75| 6.67| 12.0| 0| 1|2016|
42.0| 8010.76| 6.71| 10.0| 0| 1|2014|
37.0| 9191.58| 5.56| 9.0| 0| 1|2016|
+----+--------------+-----+---------+---------------+-----+----+
only showing top 5 rows

In [13]:
from pyspark.ml.feature import VectorAssembler

In [14]:
assembler = VectorAssembler(inputCols = ['Age',
                                         'Total_Purchase',
                                         'Account_Manager',
                                         'Years',
                                         'Num_Sites',
                                        'year'], outputCol = 'features')

In [15]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [16]:
log_regression = LogisticRegression(featuresCol='features',labelCol='Churn')

In [17]:
pipeline = Pipeline(stages=[assembler,log_regression])

In [18]:
train_data,test_data = df02.randomSplit([0.7,0.3])

In [19]:
fit_model = pipeline.fit(train_data)

In [20]:
test_result = fit_model.transform(test_data)

In [21]:
test_result.show(5)

+----+--------------+-----+---------+---------------+-----+----+--------------------+--------------------+--------------------+----------+
 Age|Total_Purchase|Years|Num_Sites|Account_Manager|Churn|year| features| rawPrediction| probability|prediction|
+----+--------------+-----+---------+---------------+-----+----+--------------------+--------------------+--------------------+----------+
25.0| 9672.03| 5.49| 8.0| 0| 0|2006|[25.0,9672.03,0.0...|[5.30139831558437...|[0.99504010422941...| 0.0|
26.0| 8787.39| 5.42| 11.0| 1| 1|2014|[26.0,8787.39,1.0...|[0.87311583569283...|[0.70539362472213...| 0.0|
28.0| 8670.98| 3.99| 6.0| 0| 0|2006|[28.0,8670.98,0.0...|[8.49965439505532...|[0.99979650271873...| 0.0|
29.0| 9378.24| 4.93| 8.0| 0| 0|2015|[29.0,9378.24,0.0...|[5.36227620837044...|[0.99533167741626...| 0.0|
29.0| 10203.18| 5.82| 8.0| 1| 0|2014|[29.0,10203.18,1....|[4.21855844253884...|[0.98549368207991...| 0.0|
+----+--------------+-----+---------+---------------+-----+----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

In [22]:
# Não há correlação de Churn com o campo Year que eu criei, então eu poderia excluir esta variável do modelo.

test_result.corr('Churn','year')

Out[422]: 0.006776573065411779

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn',metricName='areaUnderROC')

In [25]:
AUC = my_eval.evaluate(test_result)

AUC

Out[421]: 0.7510822510822511